In [10]:
import numpy as np
from sklearn.cluster import KMeans
from collections import defaultdict
from math import radians, cos, sin, asin, sqrt
import sqlite3

# -------------------------------
# 📍 Race Locations (lat, lon)
# Add or modify as needed
# -------------------------------
# Connect to the database
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Query to fetch race locations and their lat/lon
query = """
SELECT fc.circuit, fg.latitude, fg.longitude
FROM fone_calendar fc
JOIN fone_geography fg ON fc.geo_id = fg.id
WHERE fc.year = 2025
"""
cursor.execute(query)

# Build the race_locations dictionary
race_locations = {row[0]: (row[1], row[2]) for row in cursor.fetchall()}

# Close the database connection
conn.close()

track_names = list(race_locations.keys())
locations = np.array(list(race_locations.values()))

# -------------------------------
# 🌍 Haversine Distance Function
# -------------------------------
def haversine(loc1, loc2):
    lat1, lon1 = loc1
    lat2, lon2 = loc2
    # convert to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return 6371 * c  # Radius of Earth in km

# -------------------------------
# Step 1: Cluster the Races
# -------------------------------
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(locations)
labels = kmeans.labels_

clusters = defaultdict(list)
for i, label in enumerate(labels):
    clusters[label].append(track_names[i])

# -------------------------------
# Step 2: TSP within Clusters
# -------------------------------
def nearest_neighbor_tsp(track_list):
    unvisited = track_list[:]
    path = [unvisited.pop(0)]
    while unvisited:
        last = path[-1]
        next_track = min(unvisited, key=lambda t: haversine(race_locations[last], race_locations[t]))
        path.append(next_track)
        unvisited.remove(next_track)
    return path

# -------------------------------
# Step 3: Cluster Centroids
# -------------------------------
cluster_centroids = {}
for cluster_id, tracks in clusters.items():
    lats = [race_locations[t][0] for t in tracks]
    lons = [race_locations[t][1] for t in tracks]
    cluster_centroids[cluster_id] = (np.mean(lats), np.mean(lons))

# -------------------------------
# Step 4: TSP over Clusters
# -------------------------------
def tsp_on_centroids(cluster_order):
    unvisited = cluster_order[:]
    path = [unvisited.pop(0)]
    while unvisited:
        last = path[-1]
        next_cluster = min(unvisited, key=lambda c: haversine(cluster_centroids[last], cluster_centroids[c]))
        path.append(next_cluster)
        unvisited.remove(next_cluster)
    return path

cluster_order = tsp_on_centroids(list(clusters.keys()))

# -------------------------------
# Step 5: Flatten Final Schedule
# -------------------------------
final_schedule = []
for cid in cluster_order:
    intra_cluster_schedule = nearest_neighbor_tsp(clusters[cid])
    final_schedule.extend(intra_cluster_schedule)

# -------------------------------
# Done! Print Final Calendar
# -------------------------------
print("\n🏁 Optimized F1 Calendar:")
for i, race in enumerate(final_schedule, 1):
    print(f"{i:02d}. {race}")



🏁 Optimized F1 Calendar:
01. Sakhir
02. Lusail
03. Yas Island
04. Jeddah
05. Baku
06. Imola
07. Monza
08. Monaco
09. Barcelona
10. Spa-Francorchamps
11. Zandvoort
12. Silverstone
13. Spielberg
14. Budapest
15. Miami
16. Austin
17. Mexico City
18. Las Vegas
19. Montréal
20. São Paulo
21. Melbourne
22. Marina Bay
23. Shanghai
24. Suzuka


In [9]:
len(track_names)

20

In [2]:
%pip install -q numpy scikit-learn
%pip install -q matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
